In [250]:
import torch
import pickle
import random

from sklearn.decomposition import PCA



class close_words:
    def __init__(self,model,w2i,pca_1=False,n=5):
        self.model=model
        self.w2i=w2i
        self.n=n
        self.i2w=dict(zip(w2i.values(),w2i.keys()))
        pca = PCA(n_components=10)
        self.embeddings=model['embedding_in.weight'].detach() if not pca_1 else torch.tensor(pca.fit_transform(model['embedding_in.weight'].detach()))
 
    def input_word(self,word):
        self.sample_idx=self.w2i[word]
        self.sample_vec=self.embeddings[self.sample_idx]
    
    def sample_word(self):
        
        self.sample_idx=random.sample(list(self.w2i.values()),1)[0]
        self.sampled_word=self.i2w[self.sample_idx]
        self.sample_vec=self.embeddings[self.sample_idx]
        print('임의의 단어:',self.sampled_word)
    
    def l2_dist(self):
        
        trial=torch.mul((self.embeddings-self.sample_vec),(self.embeddings-self.sample_vec))
        trial[self.sample_idx]=100    
        trial=torch.sum(trial,dim=1)
        #rec=torch.argmax(-self.trial).item()
        idexes=torch.argsort(-trial, descending=True)[:self.n]
        step=1
        print('사전사이즈:',len(self.i2w))
        for idx in idexes:
            if idx==len(w2i): #unk 제거
                continue
            print(step,'번째 가까운 단어:',self.i2w[idx.item()])
            step+=1
                
    def cos_sim(self):
        
   
        upper=torch.sum(torch.mul(self.embeddings,self.sample_vec),dim=1)
  
        lower=torch.sum(torch.mul(self.embeddings,self.embeddings),dim=1)+torch.sum(torch.mul(self.sample_vec,self.sample_vec))
    
        trial=torch.div(upper,lower)

        trial[self.sample_idx]=-100
        idexes=torch.argsort(trial, descending=True)[:self.n]
        step=1
        for idx in idexes:
            if idx==len(w2i)+1: #unk 제거
                continue
            print(step,'번째 가까운 단어:',self.i2w[idx.item()])
            step+=1
        

In [251]:
#model=torch.load('./w2v_withoud_ns',map_location='cpu')
model=torch.load('./training_with_mecab_2018_2_epoch_14',map_location='cpu')['state_dict']

with open('./preprocessing//vocab_mecab.pickle','rb') as f:
    w2i=pickle.load(f)
    
print('임베딩 사이즈:',model['embedding_in.weight'].shape)  #unk 랑 embedding이 0부터인것을 몰랐음. 0은 버리면 됨
print('사전 사이즈:',len(w2i))  

임베딩 사이즈: torch.Size([30655, 256])
사전 사이즈: 30654


In [256]:
showing=close_words(model,w2i,pca_1=True,n=5)
showing.sample_word()
showing.l2_dist()
print('임의의 단어',showing.sampled_word)
showing.cos_sim()

임의의 단어: ('제목', 'NNG')
사전사이즈: 30654
1 번째 가까운 단어: ('자본', 'NNG')
2 번째 가까운 단어: ('문사', 'NNG')
3 번째 가까운 단어: ('맛보기', 'NNG')
4 번째 가까운 단어: ('철두철미', 'MAG')
5 번째 가까운 단어: ('니까요', 'VCP+EC')
임의의 단어 ('제목', 'NNG')
1 번째 가까운 단어: ('맛보기', 'NNG')
2 번째 가까운 단어: ('문사', 'NNG')
3 번째 가까운 단어: ('자본', 'NNG')
4 번째 가까운 단어: ('영양소', 'NNG')
5 번째 가까운 단어: ('익힌', 'VV+ETM')


In [268]:
showing.input_word(('과제', 'NNG'))
showing.l2_dist()
print('====')
showing.cos_sim()

사전사이즈: 30654
1 번째 가까운 단어: ('있', 'VA')
2 번째 가까운 단어: ('없', 'VA')
3 번째 가까운 단어: ('강의', 'NNG')
4 번째 가까운 단어: ('문제', 'NNG')
5 번째 가까운 단어: ('같', 'VA')
====
1 번째 가까운 단어: ('있', 'VA')
2 번째 가까운 단어: ('없', 'VA')
3 번째 가까운 단어: ('강의', 'NNG')
4 번째 가까운 단어: ('않', 'VX')
5 번째 가까운 단어: ('같', 'VA')


## UNK 빈도수

In [98]:
with open('./preprocessing/vocab_without_josa_gut_su_freq.pickle','rb') as f:
    total_freq=pickle.load(f)
    
with open('./preprocessing/vocab_without_josa_gut_su.pickle','rb') as h:
    voc=pickle.load(h)
    
not_unk=list(voc.keys())

freq_sum=0

def summing(x):
    
    global freq_sum
    freq_sum+=x

_=list(map(lambda x:summing(total_freq[x]),not_unk))

ratio=(sum(total_freq.values())-freq_sum)/sum(total_freq.values())
print(ratio)

0.014246669941204219


In [102]:
## unk 빈도수가 문제는 아닌듯??